# 1. Procesado de datos

In [11]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/train.csv')

df

,label,text,id
0,0,"Hi Roy hope you are ok, Trans people are not g...",95e98db99c2
1,0,But fuckin' hell what even is biology,a87b8708e63
2,0,Whose the nice looking clergyman?,5b3cb03803f
3,1,"AIDS ARE IN YOUR WAY, SIN HAS CONSEQUENCES AND...",5b0bad2347e
4,0,to learn,332048bd188
...,...,...,...
8143,1,Yeah...if we still alive at the time of the ne...,9141edf7189
8144,0,There is something fundamentally wrong with this,fde30735725
8145,1,"This always confused me. If homosexuality is ""...",3d3456b6659
8146,1,disgusting,eace075fe4d


In [12]:
df.isnull().sum()

label    0
text     0
id       0
dtype: int64

In [13]:
df['label'].value_counts()

label
0    5172
1    2976
Name: count, dtype: int64

In [14]:
import spacy

nlp = spacy.load('en_core_web_lg')

In [46]:
en_stopwords = nlp.Defaults.stop_words

def remove_stop_words(text):
    cleanText = ''
    phrase = nlp(text)
    for token in phrase:
        if not token.is_stop and not token.is_punct and not token.like_url:
            cleanText += ' ' + token.text

    return cleanText

df['text_cleaned'] = df['text'].apply(remove_stop_words)

In [32]:
df['text_cleaned']

0        Hi Roy hope ok Trans people gay thing s ramme...
1                                     fuckin hell biology
2                                  nice looking clergyman
3                           AIDS WAY SIN CONSEQUENCES BAD
4                                                   learn
                              ...                        
8143     Yeah alive time election happen fairly soon U...
8144                                  fundamentally wrong
8145     confused homosexuality big deal proud normal ...
8146                                           disgusting
8147     Peter Sørensen note Peter poor maths 13 27 eq...
Name: text_cleaned, Length: 8148, dtype: object